# Image set application tools

This is a demonstration how to proceed your raw data, that is holohraphy images from the TEM microscope, reconstruct phase and amplitude, make automatic alignment and save them in h5 file.

First, we import libraries


In [ ]:
import os
os.environ['FOR_DISABLE_CONSOLE_CTRL_HANDLER'] = '1'

# the followings are a local python libraries:
from align_panel import point_registration, fine_adjust
from phase import Imgset_new
from imgsetlib import Imgset

Definde our raw data. Here are 4 images: 2 images of sample and 2 reference images without sample.

In [ ]:
img0_path = 'test_data/-4-H.dm3'
ref0_path = 'test_data/-4-R2.dm3'
img1_path = 'test_data/+4-H.dm3'
ref1_path = 'test_data/+4-R2.dm3'

Now, we make new imagesets, reconstruct phase and then save data in h5 file.
The imagesets are saved in h5 file with inner hierarchy. One file correspoends to one whole experiment with several imagesets. Inside the h5 file, there are created groups, one for each imageset. Those groups are labeled by order numbers. Number 0 is for the reference imageset, other numbers (integers) are for the other imagesets. Each group contains at least 7 datasets: <br>
1) img (original image)<br>
2) ref (original reference image)<br>
3) amplitude (amplitude image)<br>
4) phase (phase image)<br>
5) unwrapped_phase (unwrapped phase image)<br>
6) img_metadata (metadata to original image)<br>
7) ref_metadata (metadata to reference image)<br>
<br>
#There are usually some warnigs, which I ingnore.

In [ ]:
"""create new imagesets"""
imgset_new0 = Imgset_new(img0_path,ref0_path); print("<<< imgset_new0 done")
imgset_new1 = Imgset_new(img1_path,ref1_path); print("<<< imgset_new1 done")

"""do phase reconstruction """
imgset_new0.phase_reconstruction()
imgset_new1.phase_reconstruction()

"""save data"""
filename = "test_data/mytestfile5.h5"
imgset_new0.save(filename,0)
imgset_new1.save(filename,1)

Lest's open this new-created file by the Imageset class. The representative object is imgset1

In [ ]:
imgset1 = Imgset("test_data/mytestfile5.h5",1)

Do autoalignment
The autoalignment can be improved by deleting the background (del_back=True) and by setting the roughness of finding the background border (low numbers leads to distubations by nois).

We also define what kind of images will be used to this alignment. Here we use amplitude (amplitude_stat and amplitude).

In [ ]:
imgset1.autoalign(1000,del_back=False, img_stat = imgset1.amplitude_stat, img_move = imgset1.amplitude)
print(">>> This is the tmat from autoalignment:"); print(imgset1.tmat)

Manual alignment is closely explained in the dedicated notebook. Here I show how to run it using the imageset, but this is supposed to be used in common script not in Jupyter notebook. For the manual alignment, I choose to use unwrapped phase, but you can choose also another kind of image. All kind of images will lead to the same result, just depens on the personal preferences.
First we run the point alignment.

In [ ]:
img_stat_manual = imgset1.unwrapped_phase_stat
img_move_manual = imgset1.unwrapped_phase

imgset1.manual_point(img_stat_manual, img_move_manual)

Now the fine alignment

In [ ]:
imgset1.manual_fine(img_stat_manual, img_move_manual, imgset1.tmat)